<a href="https://colab.research.google.com/github/yevintheenura01/DL_Assignment/blob/Yevin/pre_requirements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>